In [6]:
import parse_output
import pandas as pd
from datetime import datetime

In [2]:
def was_traceroute_successful(traceroute):
    traceroute_ips = traceroute.filter(regex=".*ip.*")
    """Returns true if we found the resolved IP in the traceroute (success)."""
    ips = list(traceroute_ips)
    resolved_ips = [ip for ip in ips if ip is not None]
    dest_ip = list(resolved_ips)[0]
    return dest_ip in resolved_ips[1:]

In [3]:
def get_final_rtt(traceroute_rtts):
    final_rtt = None
    for rtt in traceroute_rtts:
        try:
            final_rtt = float(rtt)
        except:
            pass
    assert final_rtt != None
    return final_rtt 

In [4]:
def cleanup_trees_for_rtt_analysis(trees):
    # Only consider successful rtts
    print(f'{len(trees)} total traceroutes.')
    successful_rtt_trees = trees[trees.apply(lambda x: was_traceroute_successful(x), axis=1)]

    # We only care about the rtts
    print(f'{len(successful_rtt_trees)} fully successful traceroutes.')
    rtt_trees = successful_rtt_trees.filter(regex=".*rtt.*|dest name")

    # We only want the final rtt we observed
    rtt_trees["final_rtt"] = rtt_trees.apply(lambda row: get_final_rtt(row), axis=1)

    # Simplified into two columns
    simplified_rtts = rtt_trees[["dest name", "final_rtt"]]

    return simplified_rtts

In [5]:
aws_ohio_data_filepath = "data/aws-ohio-results-1637264695.csv"
aws_ohio_trees = parse_output.parse_output(aws_ohio_data_filepath)
aws_ohio_simplified_rtts = cleanup_trees_for_rtt_analysis(aws_ohio_trees)

#aws_virginia_data_filepath = "data/"

home_victor_data_filepath = "data/home-victor-results-1637268720-incomplete.csv"
home_victor_trees = parse_output.parse_output(home_victor_data_filepath)
home_victor_simplified_rtts = cleanup_trees_for_rtt_analysis(home_victor_trees)
 # inner join the dest names to filter on
ohio_dests = pd.DataFrame(aws_ohio_simplified_rtts["dest name"].unique())
home_dests = pd.DataFrame(home_victor_simplified_rtts["dest name"].unique())
merged_dests = ohio_dests.merge(home_dests, how="inner")
aws_ohio_comparable = aws_ohio_simplified_rtts[aws_ohio_simplified_rtts.apply(lambda row: row['dest name'] in list(merged_dests[0]), axis=1)]
home_comparable = home_victor_simplified_rtts[home_victor_simplified_rtts.apply(lambda row: row['dest name'] in list(merged_dests[0]), axis=1)]
#home_comparable.to_csv("output/home-comparable.csv")
aws_ohio_comparable.to_csv("output/aws-ohio-comparable.csv")

36860 total traceroutes.
9013 fully successful traceroutes.


/tmp/ipykernel_3412/938413540.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtt_trees["final_rtt"] = rtt_trees.apply(lambda row: get_final_rtt(row), axis=1)


9665 total traceroutes.
3734 fully successful traceroutes.


In [5]:
home_victor_data_filepath = "data/home-victor-results-1637268720-incomplete.csv"
home_victor_trees = parse_output.parse_output(home_victor_data_filepath)
#home_victor_weekend = home_victor_trees[home_victor_trees['tim']]
#home_victor_simplified_rtts = cleanup_trees_for_rtt_analysis(home_victor_trees)

In [10]:
weekend_start = datetime(year=2021, month=11, day=19, hour=12).timestamp()
weekend_end = datetime(year=2021, month=11, day=22, hour=0).timestamp()

home_victor_weekend_trees = home_victor_trees[home_victor_trees["time"]]

1637557200.0